In [10]:
import numpy as np
import pandas as pd
import heapq as hp
import multiprocessing as mp
from time import sleep
from binance.client import Client

In [19]:
class StockMarket:

    
    # эта функция вызывается когда мы хотим купить amount монеток по текущей цене
    def make_buy_order(self, currency_pair, amount):
        amount *= (1 + self.fee)
        if not self.spend_till_end and self.balance[0] < amount:
            return [0, self.balance[0]]
        spent = 0
        amount = min(amount, self.balance[0])
        res_amount = amount
        for idx in range(len(self.orders['asks'])):
            if self.orders['asks'][idx][1] > amount:
                spent += self.orders['asks'][idx][0] * amount
                amount = 0
                break
            else:
                amount -= self.orders['asks'][idx][1]
                spent += self.orders['asks'][idx][1] * self.orders['asks'][idx][0]
        return [res_amount, -spent]

    
    # если хотим продать amount монеток по рыночной цене - нам сюда
    def make_sell_order(self, currency_pair, amount):
        amount 
        if not self.spend_till_end and self.balance[1] < amount:
            return [0, self.balance[1]]
        capital = 0
        amount = min(amount, self.balance[1])
        res_amount = amount
        for idx in range(len(self.orders['bids'])):
            if self.orders['bids'][idx][1] > amount:
                capital += self.orders['bids'][idx][0] * amount
                amount = 0
                break
            else:
                amount -= self.orders['bids'][idx][1]
                capital += self.orders['bids'][idx][1] * self.orders['bids'][idx][0]
        return [-res_amount, capital]
               
        
    # на вход поступает словарик {валютная пара: сколько хотим купить(если число отрицательное - продать)}
    # словарик с балансом {валюта: количество, которым мы располагаем}, словарик-стакан по всем нужным валютным 
    # парам и флаг - что мы делаем, если у нас не хватает баланса для покупки желаемого количества валюты
    # по умолчанию мы покупаем столько, сколько позволяют наши средства
    def query(self, query_dict, balance, orders, spend_till_end=True): 
        self.spend_till_end = spend_till_end
        result = dict()
        for pair in query_dict:
            self.orders = orders[pair]
            self.balance = balance[pair]
            if query_dict[pair] > 0:
                result[pair] = self.make_buy_order(pair, query_dict[pair])
            elif query_dict[pair] < 0:
                result[pair] = self.make_sell_order(pair, -query_dict[pair])
        return result # возвращаем словарик с изменениями в количестве валюты А и В
    
    
    def __init__ (self, ):
        self.bnb_fee = 0.00075
        self.fee = 0.001
        self.orders = []

In [20]:
market = StockMarket()

In [21]:
# это для тестирования, на самом деле словарик {ваоютная пара: {bids : [[price1, quantity1], ...],
# asks: [[...]]}, ...} должен предоставлять DataCather

def get_orderbook(): 
    api_key = 'E5Qcm1IZZHToGacR1W108lpmTTjp5UUiScMJEbN7sgo2f4FYj7QQQkoGvnCYdDaj'
    api_secret = 'unvyAZ4ary0vCfGRlYHlA0yDRd82N5gzfYKK8J8iqaHdL3aMnKoI9kUZXZrVEZVN'
    client = Client(api_key, api_secret)
    orders = {'btcusdt': client.get_order_book(symbol='BTCUSDT'),
                          'ltcbtc': client.get_order_book(symbol='LTCBTC')}
    for pair in orders:
        orders[pair]['bids'] = [[float(orders[pair]['bids'][i][0]),
                                 float(orders[pair]['bids'][i][1])]for i in range(len(orders[pair]['bids']))]
        orders[pair]['asks'] = [[float(orders[pair]['asks'][i][0]),
                                 float(orders[pair]['asks'][i][1])]for i in range(len(orders[pair]['asks']))]
    return orders

In [22]:
market.query({'btcusdt': 3, 'ltcbtc': -5}, {'btcusdt':[2, 1e4], 'ltcbtc':[4, 2]}, get_orderbook())

{'btcusdt': [2, -19285.48667128], 'ltcbtc': [-2, 0.015742]}